# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
flight_schedules = {"london": ["08:00", "15:00"], "paris": ["09:00", "16:00"], "tokyo": ["12:00"], "berlin": ["07:00", "13:00"]}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def get_flight_schedules(destination_city):
    print(f"Tool get_flight_hours called for {destination_city}")
    city = destination_city.lower()
    return flight_schedules.get(city, "Unknown")

def flight_confirmation_number(destination_city, date, hour):
    import random
    number = destination_city[:3].upper() + ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=6))
    print(f"Tool flight_confirmation_number called for {destination_city} on {date} at {hour}, returning {number}")
    return number

In [ ]:
print(get_ticket_price("London"))
print(get_flight_schedules("London"))
print(flight_confirmation_number("London", "2024-10-01", "15:00"))

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need \
        to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

schedule_function = {
    "name": "get_flight_schedules",
    "description": "Get the daily flight schedules (departure times) to the destination city. Call this \
        whenever you need to know the flight times, for example when a customer asks 'What time \
        are the flights to this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

confirmation_function = {
    "name": "flight_confirmation_number",
    "description": "Get a flight confirmation number for a booking. Call this whenever you need to \
        provide a confirmation number, after a customer has selected a destination city, a flight \
        date and a departure time, and also asked for the price. For example when a customer says \
        'I'd like to book that flight'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "date": {
                "type": "string",
                "description": "The date of the flight, in YYYY-MM-DD format",
            },
            "hour": {
                "type": "string",
                "description": "The departure time of the flight, in HH:MM format",
            },
        },
        "required": ["destination_city", "date", "hour"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": schedule_function},
         {"type": "function", "function": confirmation_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:
def handle_price_tool_call(tool_call_id, arguments):
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call_id
    }
    return response, city
    
def handle_schedule_tool_call(tool_call_id, arguments):
    city = arguments.get('destination_city')
    schedules = get_flight_schedules(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"schedules": schedules}),
        "tool_call_id": tool_call_id
    }
    return response, city

def handle_confirmation_tool_call(tool_call_id, arguments):
    city = arguments.get('destination_city')
    date = arguments.get('date')
    hour = arguments.get('hour')
    confirmation = flight_confirmation_number(city, date, hour)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"date": date,"hour": hour,"confirmation_number": confirmation}),
        "tool_call_id": tool_call_id
    }
    return response, city

def handle_tool_call(message):
    print("Number of tool calls:", len(message.tool_calls))
    
    tool_call = message.tool_calls[0]
    print("Tool call is for function:", tool_call.function.name)
    arguments = json.loads(tool_call.function.arguments)
    
    if tool_call.function.name == "get_ticket_price":
        response, city = handle_price_tool_call(tool_call.id, arguments)
    elif tool_call.function.name == "get_flight_schedules":
        response, city = handle_schedule_tool_call(tool_call.id, arguments)
    elif tool_call.function.name == "flight_confirmation_number":
        response, city = handle_confirmation_tool_call(tool_call.id, arguments)
    
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()